In [1]:
from data_utils import TgsDataset
from resunet import ResUnet
from metric import metric
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch
import random
import os
import torch.nn as nn

In [2]:
batch_size = 40
train_df = pd.read_csv('tgs/train.csv')
test_df = pd.read_csv('tgs/sample_submission.csv')
train_df, val_df = train_test_split(train_df,test_size=0.2,random_state=69)
tgs_train_dataset = TgsDataset(train_df,'tgs/train/images','tgs/train/masks')
tgs_val_dataset = TgsDataset(val_df,'tgs/train/images','tgs/train/masks')
tgs_test_dataset = TgsDataset(test_df,'tgs/test/images')
tgs_train_loader = DataLoader(tgs_train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
tgs_val_loader = DataLoader(tgs_val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
tgs_test_loader = DataLoader(tgs_test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [3]:
#for i, sample_batch in enumerate(tgs_loader):
#    if i == 1:
#        print(sample_batch['image']*255)
#        break

In [4]:
#plt.figure(figsize=(10,10))
#for i, sample_batch in enumerate(tgs_loader):
#    if i == 3:
#        for g in range(batch_size):
#            plt.subplot(batch_size,2,g*2+1)
#            plt.imshow(np.transpose(sample_batch['image'][g].numpy(),(1,2,0)))
#            plt.subplot(batch_size,2,g*2+2)
#            plt.imshow(sample_batch['mask'][g].numpy())
#        break
#plt.tight_layout()
#plt.show()

In [5]:
device = torch.device('cuda')
dtype = torch.float32
#model = Unet([3,16,32,64,128,256]).to(device=device,dtype=dtype)

In [6]:
#for i, param in enumerate(model.parameters()):
#    if i == 1:
#        print(param)

In [ ]:
best_val_loss = 10
for n in range(20):
    lr = 5e-2
    decay = 0
    break_num = 0
    local_val_loss = 10
    model = ResUnet([3,16,32,64,128,256]).to(device=device,dtype=dtype)
    for e in range(8000):
        optimizer = optim.Adam(model.parameters(), lr=lr)
        for i, sample_batch in enumerate(tgs_train_loader):
            images = sample_batch['image'].to(device=device, dtype=dtype)
            targets = sample_batch['mask'].view(40,1,128,128).to(device=device, dtype=dtype)
            outputs = model(images)
            #print(outputs.shape)
            #print(targets.shape)
            train_loss = F.binary_cross_entropy(outputs,targets)
            #if i % 20 == 0:
            #    print(loss)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            #with torch.no_grad():
            #    for k, w in enumerate(model.parameters()):
            #        if k == 1:
            #            print(w)
            #            print(w.grad)
            #        w -= lr * w.grad
                
            #       Manually zero the gradients after running the backward pass
            #        w.grad.zero_()
    
        val_loss = 0
        iou = 0
        with torch.no_grad():
            for j, sample_batch in enumerate(tgs_val_loader):
                images = sample_batch['image'].to(device=device, dtype=dtype) #/ 1000000
                targets = sample_batch['mask'].view(40,1,128,128).to(device=device, dtype=dtype)
                outputs = model(images)
                out = torch.zeros_like(outputs)
                out[outputs>0.5] = 1
                tar = torch.zeros_like(targets)
                tar[targets>0.5] = 1
                val_loss += F.binary_cross_entropy(out,tar)
                iou += metric(out,tar)
            val_loss /= 20
            iou /= 20
    
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model,'tgs_best_res_3.pth')
            print('update model!! val_loss is %.4f , iou is %.4f' % (val_loss, iou))
        
        if val_loss < local_val_loss:
            local_val_loss = val_loss
            decay = 0
            break_num = 0
        else:
            decay += 1
            break_num += 1
            
        if decay > 4:
            lr /= 2
            decay = 0
        
        if break_num > 10:
            break
    
    print('num : %d    train_loss : %.4f    val_loss : %.4f    iou : %.4f' % (n, train_loss, val_loss, iou))
    print('\n')
    print('\n')
    
        

F:\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResUnet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
F:\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
F:\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
F:\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type EncoderBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
F:\Anaconda3\lib\site-packages\torch\serialization.p

update model!! val_loss is 6.7862 , iou is 0.1370
update model!! val_loss is 5.0391 , iou is 0.3605
update model!! val_loss is 4.9344 , iou is 0.3968
update model!! val_loss is 4.1449 , iou is 0.4851
update model!! val_loss is 4.1017 , iou is 0.4830
update model!! val_loss is 3.9571 , iou is 0.4992
update model!! val_loss is 3.6678 , iou is 0.5308


In [ ]:
#for i, param in enumerate(model.parameters()):
#    if i == 1:
#        print(param)

In [ ]:
#for i, sample_batch in enumerate(tgs_loader):
#    if i == 1:
#        images = sample_batch['image'].to(device=device, dtype=dtype) / 1000000
#        mask = model(images)
#        out = torch.zeros_like(mask)
#        out[mask>0.5] =1
#        plt.figure(figsize=(10,40))
#        for j in range(10):
#            plt.subplot(10,3,j*3+1)
#            plt.imshow(np.transpose(sample_batch['image'][j].numpy(),(1,2,0)))
#            plt.subplot(10,3,j*3+2)
#            plt.imshow(sample_batch['mask'][j].numpy())
#            plt.subplot(10,3,j*3+3)
#            plt.imshow(out[j].view(128,128).to(device=torch.device('cpu')).numpy())
#        plt.tight_layout()
#        plt.show()
#        break